## Import Libraries

In [ ]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml import Pipeline
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.regression import LSTM

## PySpark Session

In [ ]:
spark = SparkSession.builder.appName("SparkApp").getOrCreate()

## Load data

In [ ]:
data = spark.read.format("csv").option("header", "true").load("my_file.csv")

## Create feature vector [f1, f2, f3] as input of model

In [ ]:
featureColumns = ["f1", "f2", "f3"]
assembler = VectorAssembler(inputCols=featureColumns, outputCol="features")
data = assembler.transform(data)